# Basic usage of Ctez and Checker

In this notebook, we're going to show how to compile, deploy and test the Checker library using Python and the [PyTezos](pytezos.org/) library.

We assume that the `checker` package was installed in the current environment, that the LIGO compiler is available, as a command with the name `ligo`, and that a Tezos RPC node is available at the following url:

In [ ]:
TEZOS_RPC="http://localhost:20000"



As this notebook is versioned, we recommend duplicating it before any modification.

Let's define a few helpers for PyTezos:

In [ ]:
import pytezos as tz
import time
from checker_tools.client.operations import inject

alice = tz.Key.from_encoded_key("edsk3QoqBuvdamxouPhin7swCvkQNgq4jP5KZPbwWNnwdZpSpJiEbq")

In [ ]:
alice.public_key_hash()

In [ ]:
ptz = tz.pytezos.using(TEZOS_RPC, alice)

In [ ]:
ptz.balance()

In [ ]:
def do_it(call, amount=None):
    """Calls an entrypoint, optionally sending an amount of mutez"""
    if amount is not None:
            call = call.with_amount(amount)
    tx = call.as_transaction()
    inject(ptz, tx.autofill(ttl=None).sign())
    
def add_operator_fa2(owner, address, token, token_id):
    return do_it(token.update_operators([{
        "add_operator": {
            "owner": owner.public_key_hash(),
            "operator": address,
            "token_id": token_id
        }
    }]))

def add_allowance_fa12(token, spender, quantity):
    return do_it(token.approve({
        "spender": spender.address,
        "value": quantity
        })
    )

def now_plus_10():
    return int(time.time() + 10)

## Building and deploying

In [ ]:
from checker_tools.client.compilation import compile_everything
from checker_tools.builder.config import CheckerRepo
from checker_tools.client import checker

Note that you probably won't be able to compile from a parent directory with a dockerized `ligo`.

In [ ]:
conf = CheckerRepo("..")
conf.default_config


In [ ]:
compile_everything(out_dir="generated/michelson",
                   src_dir=conf.src,
                   vendor_dir="../vendor")

Next, we can deploy our contracts, for later use.

In [ ]:
ctez_contracts = checker.deploy_ctez(ptz, conf)
oracle = checker.deploy_contract(ptz, source_file="../utils/mock_oracle.tz",
                                 initial_storage={'owner': alice.public_key_hash(),
                                                  'price': (1, 1)})
cfmm = checker.deploy_contract(ptz, source_file="../utils/mock_cfmm_oracle.tz",
                               initial_storage={'owner': alice.public_key_hash(),
                                                'price': (1, 1)})
wtez = checker.deploy_wtez(ptz, conf)
wctez = checker.deploy_wctez(ptz,
                             conf,
                             ctez_fa12_address=ctez_contracts["fa12_ctez"].address)

And now, Checker. Checker is a big contract, made of several entrypoints that are independently deployed to a big map (sometimes in several chunks).

Initially, Checker is deployed in an "Unsealed" state; in this state, it isn't possible to use the contract, but it is possible to deploy the various entrypoints.

Once every entrypoint is deployed, it is possible to seal the contract.

In [ ]:
ch = checker.deploy_checker(ptz, conf, oracle=oracle, collateral_fa2=wtez,
                       cfmm_token_fa2=wctez, ctez_cfmm=ctez_contracts["cfmm"])

In [ ]:
ch.storage()